In [36]:
from flask import Flask, request, jsonify
from torchvision import transforms
from PIL import Image
import numpy as np
import torch
from ultralytics import YOLO
import os

app = Flask(__name__)

# Load the pre-trained YOLO model
model = YOLO('best.pt')

# Define classes
classes = ['mitosis', 'normal']

@app.route('/predict', methods=['GET'])
def predict():
    # Get the image filename from the request query parameter
    filename = request.args.get('image_filename', '')
    
    # Validate the image_filename
    if not filename:
        return jsonify({'error': 'image_filename parameter is missing.'}), 400
    
    # Read the image file
    base_path = r"mitovision\storage\app\public\images"
    
    image_path = os.path.join(base_path, filename)

    image_path = image_path.replace('\\', '/')
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        return jsonify({'error': f'Error reading the image file: {image_path}'}), 400
    
    # Perform prediction
    with torch.no_grad():
        outputs = model(image)

    # Get prediction result
    prediction = classes[outputs[0].probs.data.argmax()]
    print("Predicted class:", prediction)
    
    # Return prediction result
    return jsonify({'prediction': prediction, 'image_filename': filename})

if __name__ == '__main__':
    try:
        app.debug = True
        app.run(host='0.0.0.0', use_reloader=False, port=5001)
    except Exception as e:
        print(f"Exception Occurred: {e}")


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.10.19:5001
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2024 22:23:27] "POST /predict HTTP/1.1" 405 -



0: 128x128 Normal 0.83, Mitosis 0.17, 103.3ms
Speed: 5.7ms preprocess, 103.3ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)


127.0.0.1 - - [20/Apr/2024 22:24:19] "GET /predict?image_filename=wUJ4MIOaaxtdOA7SKG9LRXz6u2HkIJDeZDKkpOP2.png HTTP/1.1" 200 -


Predicted class: normal

0: 128x128 Normal 0.83, Mitosis 0.17, 113.9ms
Speed: 6.0ms preprocess, 113.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)


...

Predicted class: normal

0: 128x128 Normal 0.86, Mitosis 0.14, 83.8ms
Speed: 62.1ms preprocess, 83.8ms inference, 1.1ms postprocess per image at shape (1, 3, 128, 128)


127.0.0.1 - - [20/Apr/2024 22:37:49] "GET /predict?image_filename=49NA71gLbYQof9jZFTLBWorovjWJ9f9enz8CA7Gm.jpg HTTP/1.1" 200 -


Predicted class: normal

0: 128x128 Normal 0.79, Mitosis 0.21, 83.9ms
Speed: 55.3ms preprocess, 83.9ms inference, 2.1ms postprocess per image at shape (1, 3, 128, 128)


127.0.0.1 - - [20/Apr/2024 22:38:23] "GET /predict?image_filename=sIW9Zrj5VTuldieWAmuZAOmEy8WVmPeJBqaWIcNf.jpg HTTP/1.1" 200 -


Predicted class: normal
